In [1]:
import sys
sys.path.append("../base")

import boilerplate
from pyrtl import *

import z3
z3.set_param(proof=True)

In [2]:
reset_working_block()

pc = Register(name='pc', bitwidth=4)
sp = Register(name='sp', bitwidth=4)
mem = MemBlock(name='mem', bitwidth=4, addrwidth=4)
rom = RomBlock(name='rom', bitwidth=5, addrwidth=4, romdata=[0x15, 0x18, 0, 0xf])

out = Output(name='out', bitwidth=4)

In [3]:
instr = rom[pc]
out <<= mem[sp]

with conditional_assignment:
    with instr[4] == 1:   # PUSH
        mem[sp] |= instr[0:4]
        sp.next |= sp + 1
        pc.next |= pc + 1
    with instr == 0:      # POP
        sp.next |= sp - 1
        pc.next |= pc + 1

In [4]:
sim = Simulation()
for i in range(10):
    sim.step({})

In [5]:
sim.tracer.render_trace()

<IPython.core.display.Javascript object>

In [6]:
sim.tracer.print_trace()

--- Values in base 10 ---
out 0 0 0 8 8 8 8 8 8 8
pc  0 1 2 3 3 3 3 3 3 3
sp  0 1 2 1 1 1 1 1 1 1


In [7]:
from circuit import net_to_smt

wires, ops, tr = net_to_smt(working_block(), mems=[mem])

ModuleNotFoundError: No module named 'circuit'

In [8]:
from presentation_forms import vertically
vertically(tr)

NameError: name 'tr' is not defined

In [9]:
# you can find this implementation in `verification_utils` as well
def mk_bv_array(bitwidth, addrwidth, data):
    a = z3.K(z3.BitVecSort(addrwidth), z3.BitVecVal(0, bitwidth))
    for i, d in enumerate(data):
        a = z3.Store(a, i, d)
    return a

In [10]:

from verification_utils import CHCs, mk_bv_array

state_vars = [wires.lookup(v) for v in ['pc', 'sp', 'mem']]
Inv = z3.Function("Inv", *(v.sort() for v in state_vars), z3.BoolSort())

vrom = wires.lookup_mem('rom')
crom = mk_bv_array(5, 4, [0x15, 0x18, 0x0, 0xf])

def pre_post():
    pc, sp, mem = state_vars
    out = mem[sp]

    pre = z3.And(pc == 0, sp == 0, mem == mk_bv_array(4, 4, []))
    post = z3.Implies(crom[pc] == 0xf, out == 8)

    return pre, post

def create_rules():
    from z3 import Implies, And, Or, Not
    
    pre, post = pre_post()
        
    rom_eq = (vrom == crom)
    
    sigma = state_vars
    sigma_p = [ops.primed(v) for v in state_vars]
    
    start = Implies(pre, Inv(sigma))
    step = Implies(And(Inv(sigma), rom_eq, *tr), Inv(sigma_p))
    end = Implies(And(Inv(sigma), Not(post)), False)
        
    return CHCs([start, step, end])

rules = create_rules()
rules

NameError: name 'wires' is not defined

In [11]:
s = rules.create_solver()
%time res = s.check()
res

NameError: name 'rules' is not defined

_If the result is_ `sat`, _this will display the inferred loop invariant_

In [12]:
s.model()      if res == z3.sat else None

NameError: name 'res' is not defined

_If the result is_ `unsat`, _a counterexample trace can be extracted from the proof of unsatisfiability._

In [13]:
from verification_utils import HyperResolutionProof
HyperResolutionProof(s.proof()).to_roadmap()         if res == z3.unsat else None

NameError: name 'res' is not defined